Jet matching notebook for JMENANO. Plotting distributions from MC using processor

In [8]:
#Probably not needed to run this cell, but maybe good to do? Used bleeding edge with 16 GB as configuration
!pip install --user coffea

In [1]:
import bokeh
import time
import copy
import scipy.stats as ss
from scipy.optimize import curve_fit
from coffea import hist, processor, nanoevents, util
from coffea.nanoevents.methods import candidate
from coffea.nanoevents import NanoAODSchema, BaseSchema

import awkward as ak
import numpy as np
import glob as glob
import itertools
import pandas as pd
from numpy.random import RandomState

#from dask.distributed import Client
import inspect
import matplotlib.pyplot as plt

#from lpcjobqueue import LPCCondorCluster

In [2]:
class JMENanoAODSchema(NanoAODSchema):
    """JMENano schema builder

    JMENano is an extended NanoAOD format that includes various jet collections down to low pt for JME studies
    More info at https://twiki.cern.ch/twiki/bin/viewauth/CMS/JMECustomNanoAOD
    Customization at https://github.com/nurfikri89/cmssw/blob/master/PhysicsTools/NanoAOD/python/custom_jme_cff.py
    """

    mixins = {
        **NanoAODSchema.mixins,
        "JetCalo": "Jet",
        "JetPuppi": "Jet",
        "FatJetForJEC": "Jet",
        "FatJetCHS": "Jet",
    }
    all_cross_references = {
        **NanoAODSchema.all_cross_references,
        "FatJetForJEC_genJetIdx": "GenJetAK8ForJEC",
        "FatJetCHS_genJetIdx": "GenJetAK8ForJEC",
        "JetCalo_genJetIdx": "GenJet",
        "JetPuppi_genJetIdx": "GenJet",
    }


### Import processor

In [3]:
from CoffeaJERCProcessor_PUPPI_Cleaned import Processor

In [4]:
xrootdstr = '/eos/cms/store/group/phys_jetmet/kirschen/JMENANO_EarlyDataTest/'
#adapt this to eos...phys_jetmet

#rootfiles = open('dataset_local.txt').read().split()
#rootfiles = open('dataset_local_Epsilon.txt').read().split()
#rootfiles = open('dataset_local_Premix.txt').read().split()
rootfiles = open('dataset_EOS_DY.txt').read().split()

fileslist = [xrootdstr + file for file in rootfiles]



In [5]:
fileslist

['/eos/cms/store/group/phys_jetmet/kirschen/JMENANO_EarlyDataTest/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8_RunIISummer20UL18NanoAODv9-20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1_NANOAODSIM/0178718B-3FD4-354A-BC4C-C11B16887212.root',
 '/eos/cms/store/group/phys_jetmet/kirschen/JMENANO_EarlyDataTest/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8_RunIISummer20UL18NanoAODv9-20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1_NANOAODSIM/07207868-CAF8-A545-A697-F5D189ABC029.root',
 '/eos/cms/store/group/phys_jetmet/kirschen/JMENANO_EarlyDataTest/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8_RunIISummer20UL18NanoAODv9-20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1_NANOAODSIM/082CFEB7-53F5-E841-87CC-FF519517F623.root',
 '/eos/cms/store/group/phys_jetmet/kirschen/JMENANO_EarlyDataTest/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8_RunIISummer20UL18NanoAODv9-20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1_NANOAODSIM/17C242A8-9ECF-2E4B-8A80-EE072D7715D1.roo

In [9]:
#process just two files for now
fileslist = fileslist[:2]
fileslist

['/eos/cms/store/group/phys_jetmet/kirschen/JMENANO_EarlyDataTest/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8_RunIISummer20UL18NanoAODv9-20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1_NANOAODSIM/0178718B-3FD4-354A-BC4C-C11B16887212.root',
 '/eos/cms/store/group/phys_jetmet/kirschen/JMENANO_EarlyDataTest/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8_RunIISummer20UL18NanoAODv9-20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1_NANOAODSIM/07207868-CAF8-A545-A697-F5D189ABC029.root']

In [ ]:
tstart = time.time()

outputs_unweighted = {}

seed = 1234577890
prng = RandomState(seed)
Chunk = [10000, 10] # [chunksize, maxchunks]

filesets = {'QCD': fileslist}

for name,files in filesets.items(): 
    chosen_exec = 'futures'
    output = processor.run_uproot_job({name:files},
                                          treename='Events',
                                          processor_instance=Processor(),
                                          #executor=processor.iterative_executor,
                                            executor=processor.futures_executor,
                                          executor_args={
                                              'skipbadfiles':False,
                                              'schema': JMENanoAODSchema, #NanoAODSchema, #BaseSchema
                                              'workers': 2},
                                          chunksize=Chunk[0])#, maxchunks=Chunk[1])

elapsed = time.time() - tstart
outputs_unweighted[name] = output
print(output)
#util.save(output, 'CoffeaJERCOutputs_binned_DY_WithoutDZCut.coffea')
#util.save(output, 'CoffeaJERCOutputs_binned_DY_DZCut.coffea')
util.save(output, 'CoffeaJERCOutputs_binned_something.coffea')


outputs_unweighted[name] = output
print(name + ' unweighted output loaded')
elapsed = time.time() - tstart

dict_keys(['Summer20UL18_V2_MC_L2Relative_AK4PFPuppi'])
['Summer20UL18_V2_MC_L2Relative_AK4PFPuppi']



Processing:   0%|          | 0/98 [00:00<?, ?chunk/s]

### Load coffea output file

In [ ]:
#output = util.load('CoffeaJERCOutputs_binned_DY_WithoutDZCut.coffea')
#output = util.load('CoffeaJERCOutputs_binned_DY_DZCut.coffea')
output = util.load('CoffeaJERCOutputs_binned_something.coffea')
print(output)
print ([a for a in output])

In [ ]:
# define gaussian function
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

In [ ]:
ptbins = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 20, 23, 27, 30, 35, 40, 45, 57, 72, 90, 120, 
        150, 200, 300, 400, 550, 750, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 10000 ]





In [ ]:
etabins =   [-4.889,  -4.716,  -4.538,  -4.363,  -4.191,  -4.013,  -3.839,  -3.664,  -3.489,
           -3.314,  -3.139,  -2.964,  -2.853,  -2.65,  -2.5,  -2.322,  -2.172,  -2.043,  -1.93,  -1.83,
           -1.74,  -1.653,  -1.566,  -1.479,  -1.392,  -1.305,  -1.218,  -1.131,  -1.044,  -0.957,  -0.879,
           -0.783,  -0.696,  -0.609,  -0.522,  -0.435,  -0.348,  -0.261,  -0.174,  -0.087,  0,  0.087,  0.174,
           0.261,  0.348,  0.435,  0.522,  0.609,  0.696,  0.783,  0.879,  0.957,  1.044,  1.131,  1.218,
           1.305,  1.392,  1.479,  1.566,  1.653,  1.74,  1.83,  1.93,  2.043,  2.172,  2.322,  2.5,  2.65,
           2.853,  2.964,  3.139,  3.314,  3.489,  3.664,  3.839,  4.013,  4.191,  4.363,  4.538,  4.716,
           4.889, ]#5.191 ]

In [ ]:
jetpt_length = len(output['jetpt'].axis('pt')[1:-1])
jeteta_length = len(output['jeteta'].axis('jeteta')[1:-1])
jeteta_length = len(etabins)

mean = np.zeros((jetpt_length, jeteta_length))
median = np.zeros((jetpt_length, jeteta_length))
width = np.zeros((jetpt_length, jeteta_length))
idx = []

In [ ]:
print(ptbins)
print(len(ptbins))
bins = [10,15,20,30]
#my_slices = tuple(slice(x) for x in bins)
print(ptbins[10:20])

In [ ]:
import coffea, uproot3, numpy
import mplhep
plt.style.use(mplhep.style.ROOT)
def createEfficiencyHistos(denominatorname,numeratorhistonames):
    for k in range(len(etabins)-1):
        etaBin = hist.Interval(etabins[k], etabins[k+1])                                                                                                                                                                                                     
        eta_string = 'eta_{:0>6.3f}_to_{:0>6.3f}'.format(etaBin.lo,etaBin.hi)
        eta_string = eta_string.replace('.','_')
        dhisto = output[denominatorname].integrate('jeteta', etaBin).integrate('dataset')
        print(dhisto)
        nhistos = [output[nname].integrate('jeteta', etaBin).integrate('dataset') for nname in numeratorhistonames]
        
        #ax = hist.plot1d(dhisto)
        #ax2 = ax.twinx()
        color = 'navajowhite'
        ax = hist.plot1d(dhisto, fill_opts={'color':color})
        ax.set_ylabel('Total counts', color=color)  # we already handled the x-label with ax1
        #ax2.plot(t, data2, color=color)
        ax.set_xlim(10,120)
        ax.tick_params(axis='y', labelcolor=color)
        ax2 = ax.twinx()
        for idx,nhist in enumerate(nhistos):
            print("adding",numeratorhistonames[idx])
            hist.plotratio(num=nhist,denom=dhisto, ax =ax2, clear=False,
                                #error_opts={'color': 'k', 'marker': '.'},
                                error_opts={'marker': '.'},
#                                unc='clopper-pearson',label=numeratorhistonames[idx])
                                unc='num',label=numeratorhistonames[idx])
#                            unc='poisson-ratio',label=numeratorhistonames[0])
#        hist.plotratio(num=nhistos[1],denom=dhisto, ax =ax2, clear=False,
#                            error_opts={'marker': '.'},
#                            unc='num',label=numeratorhistonames[1])
#                            unc='poisson-ratio',label=numeratorhistonames[1])
        ax2.set_title("ratiovs_{}_{}".format(denominatorname,eta_string))
        ax2.set_ylabel('Efficiency')
        ax2.set_ylim(0,1.1) 
        leg = ax2.legend()

        plt.show()

        plt.savefig("plots/ratiovs_{}_for_{}_{}.pdf".format(denominatorname,"_".join(numeratorhistonames),eta_string))
        plt.savefig("plots/ratiovs_{}_for_{}_{}.png".format(denominatorname,"_".join(numeratorhistonames),eta_string))
createEfficiencyHistos('GenJetCounts', ['GenJetCountsMatchedCHS','GenJetCountsMatchedPUPPI'])
createEfficiencyHistos('GenJetCounts', ['GenJetCountsWithDZCut'])#,'GenJetCountsMatchedCHS'])


In [ ]:
import coffea, uproot3, numpy
def dumpHistos(histoname):
    xvals = output[histoname].axis('ptresponse' if 'ptresponse' in histoname else 'pt').centers()
    fout = uproot3.create("plots/{}_export.root".format(histoname))

    f_xvals = np.linspace(0,5,5001)
    j = 0
    fewptbins = [10,15,20,30,32]
    for i in range(len(ptbins)-1):
    #for i in fewptbins:

        ptBin = hist.Interval(ptbins[i], ptbins[i+1])
        print('pt bin '+str(ptBin))

        if not 'inf' in str(ptBin):
            #pt_string = '_pT'+str(int(ptBin.lo))+'to'+str(int(ptBin.hi))                                                                                                                                                                                                     
            pt_string = '_pT_{:0>6}_to_{:0>6}'.format(int(ptBin.lo),int(ptBin.hi))
        else:
            pt_string = '_pT'+str(ptBin.lo) + 'to' + str(ptBin.hi)
            pt_string = pt_string.replace('.0','').replace('-infto','0to')

        for k in range(len(etabins)-1):

            etaBin = hist.Interval(etabins[k], etabins[k+1])
            #eta_string = '_eta'+str(etaBin.lo)+'to'+str(etaBin.hi)                                                                                                                                                                                                           
            eta_string = '_eta_{:0>6.3f}_to_{:0>6.3f}'.format(etaBin.lo,etaBin.hi)
            eta_string = eta_string.replace('.','_')



            histo = output[histoname].integrate('jeteta', etaBin).integrate('pt', ptBin) if 'ptresponse' in histoname else output[histoname].integrate('jeteta', etaBin) 
            if i==0 or histoname!="GenJetCounts": fout["{}_{}_{}".format(histoname,pt_string if 'ptresponse' in histoname else "",eta_string)] = coffea.hist.export1d(histo.integrate('dataset'))
            histvals = np.repeat(histo.axis('ptresponse' if 'ptresponse' in histoname else 'pt').centers(), np.array(histo.values()[('QCD',)],dtype='int'))

            yvals = histo.values()[('QCD',)]



            try:
                p, arr = curve_fit(gauss, xvals, yvals, p0=[10,1,1])
            except:
                continue


            fgaus = gauss(f_xvals, *p)

    #         median[i,k] = f_xvals[fgaus == np.max(fgaus)]                                                                                                                                                                                                                   
            median[i,k] = np.median(histvals)
            mean[i,k] = p[1]
            width[i,k] = p[2]
            idx.append(i)

            if(etabins[k]==2.853 or etabins[k]==0.0):
                h = np.max(histo.values()[('QCD',)])
                ax = hist.plot1d(histo, overlay='dataset')
                ax.set_title("{}_{}_{}".format(histoname,pt_string,eta_string))
     #         plt.plot(f_xvals, fgaus)                                                                                                                                                                                                                                       
                plt.text(4,0.75*h,'Mean {0:0.2f}'.format(p[1]))
                plt.text(4,0.7*h,'Median {0:0.2f}'.format(np.median(histvals)))
                plt.text(4,0.65*h,'Width {0:0.2f}'.format(p[2]))
                #plt.text(4,0.65*h,'Width {0:0.2f}'.format(p[2]))

                plt.xscale("linear") if 'ptresponse' in histoname else plt.xscale("log")
                plt.savefig("plots/{}_{}_{}.pdf".format(histoname,pt_string,eta_string))
                plt.savefig("plots/{}_{}_{}.png".format(histoname,pt_string,eta_string))
                plt.show()
    fout.close()


In [ ]:
histos= ['GenJetCountsWithDZCut','GenJetCountsMatchedPUPPI','GenJetCountsMatchedCHS','GenJetCounts','ptresponse', 'corrected_ptresponse', 'CHSptresponse', 'CHScorrected_ptresponse', 'CHSPUPPIptresponse', 'CHSPUPPIcorrected_ptresponse']
#histos= ['ptresponse', 'corrected_ptresponse', 'CHSptresponse', 'CHScorrected_ptresponse', 'CHSPUPPIptresponse', 'CHSPUPPIcorrected_ptresponse']
for histo in histos:
    dumpHistos(histo)


# Main part done here
Keep later parts (from original JERCCoffea code) for reference. Might be useful for later data/MC double-ratios

In [ ]:
histo = output['ptresponse'].integrate('jeteta', hist.Interval(-0.5, 0)).integrate('pt', hist.Interval(180, 200))

histo.values()

In [ ]:
        h = np.max(histo.values()[('QCD',)])
        ax = hist.plot1d(histo, overlay='dataset')
#         plt.plot(f_xvals, fgaus)
        plt.savefig('ptResponse'+pt_string+eta_string+'.png')
        plt.text(4,0.75*h,'Mean {0:0.2f}'.format(p[1]))
        plt.text(4,0.7*h,'Median {0:0.2f}'.format(np.median(histvals)))
        plt.text(4,0.65*h,'Width {0:0.2f}'.format(p[2]))

        plt.show()

In [ ]:
14000/h

In [ ]:
np.max(histo.values()[('QCD',)])

In [ ]:
arr = np.repeat(histo.axis('ptresponse').centers(), np.array(histo.values()[('QCD',)],dtype='int'))

In [ ]:
np.array(histo.values()[('QCD',)],dtype='int')

In [ ]:
data = {str(ptBin):mean[i] for i, ptBin in enumerate(output['jetpt'].axis('pt')[1:-1])}
# data = {str(ptBin):mean[i] for i, ptBin in enumerate(ptbins)}

data['etaBins'] = [str(etaBin) for etaBin in output['jeteta'].axis('jeteta')[1:-1]]
# data['etaBins'] = [str(etaBin) for etaBin in etabins]

df = pd.DataFrame(data=data)
df = df.set_index('etaBins')
df.to_csv('EtaBinsvsPtBinsMean.csv')

In [ ]:
data_width = {str(ptBin):width[i] for i, ptBin in enumerate(output['jetpt'].axis('pt')[1:-1])}
# data_width = {str(ptBin):width[i] for i, ptBin in enumerate(ptbins)}

data_width['etaBins'] = [str(etaBin) for etaBin in output['jeteta'].axis('jeteta')[1:-1]]
# data_width['etaBins'] = [str(etaBin) for etaBin in etabins]

df_width = pd.DataFrame(data=data_width)
df_width = df_width.set_index('etaBins')
df_width.to_csv('EtaBinsvsPtBinsWidth.csv')

In [ ]:
data_median = {str(ptBin):median[i] for i, ptBin in enumerate(output['jetpt'].axis('pt')[1:-1])}
# data_median = {str(ptBin):median[i] for i, ptBin in enumerate(ptbins)}

data_median['etaBins'] = [str(etaBin) for etaBin in output['jeteta'].axis('jeteta')[1:-1]]
# data_median['etaBins'] = [str(etaBin) for etaBin in etabins]

df_median = pd.DataFrame(data=data_median)
df_median = df_median.set_index('etaBins')
df_median.to_csv('EtaBinsvsPtBinsMedian.csv')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

In [ ]:
df

In [ ]:
# df[pt bin][eta bin]

ptBin = '300'
etaBin = '0.783'

print('mean   =', np.round(df[ptBin][etaBin],3))
print('median =', np.round(df_median[ptBin][etaBin],3))
print('width  =', np.round(df_width[ptBin][etaBin],3))

In [ ]:
# df[pt bin][eta bin]

ptBin = '300'
etaBin = '0.783'

print('mean   =', df[ptBin][etaBin])
print('median =', df_median[ptBin][etaBin])
print('width  =', df_width[ptBin][etaBin])

### Read csv


 

format for example $ 20 \text{ GeV} < p_T < 25 \text{ GeV} $ and $ 3.5 < \eta < 4.0 $


```
df = pd.read_csv('EtaBinsvsPtBinsMean.csv).set_index('etaBins')
ptBin='[20, 25)'
etaBin='[3.5, 4)'
mean = df[ptBin][etaBin]
```




In [ ]:
df_csv = pd.read_csv('EtaBinsvsPtBinsMean.csv').set_index('etaBins')

In [ ]:
df_csv

In [ ]:
df_median

In [ ]:
eff = np.random.rand(10,10)
ptbins = np.random.randint(9, size=10)
etabins = np.random.randint(9, size=10)